In [270]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import accuracy_score

# Data

In [271]:
datapath = "dataset/"

df = pd.read_csv(datapath + 'data.csv')

In [272]:
df.head()

,hPossesion,aPossesion,hshotsOnTarget,ashotsOnTarget,hshots,ashots,hfouls,afouls,hsaves,asaves,label
0,24,76,2,4,14,17,11,13,2,1,2
1,80,20,3,1,5,2,1,3,0,1,1
2,57,43,4,2,12,11,10,12,1,2,1
3,47,53,5,3,13,11,7,10,1,3,0
4,72,28,5,4,21,6,8,13,4,5,0


In [273]:
df.shape

(1272, 11)

# Nueral Net From Scratch

### Selected Data to Train

In [274]:
sample_df = df.sample(n=100, random_state=42)
sample_df.head()

,hPossesion,aPossesion,hshotsOnTarget,ashotsOnTarget,hshots,ashots,hfouls,afouls,hsaves,asaves,label
208,50,50,7,0,11,2,7,7,0,3,1
966,45,55,1,3,9,10,20,18,3,1,0
714,37,63,2,1,8,10,13,10,1,1,1
1192,64,36,15,3,20,6,15,13,1,7,0
584,28,72,0,5,3,22,12,4,3,0,2


In [275]:
train, test = np.split(sample_df.sample(frac=1, random_state=42), [int(0.8 * len(sample_df))])

train.shape, test.shape

c:\Users\Public\Anaconda\envs\dsde-cp\Lib\site-packages\numpy\_core\fromnumeric.py:57: FutureWarning: 'DataFrame.swapaxes' is deprecated and will be removed in a future version. Please use 'DataFrame.transpose' instead.
  return bound(*args, **kwds)


((80, 11), (20, 11))

In [276]:
X = train.drop(columns=['label'])
y = train['label']

X_test = test.drop(columns=['label'])
y_test = test['label']

X.shape, y.shape

((80, 10), (80,))

## init weights

In [277]:
W = np.random.randn(10, 3)
W

array([[-1.37122197,  2.34245829, -1.39733078],
       [ 0.24470155, -0.65113743,  0.86604253],
       [ 0.18300821, -1.33780026, -2.19904837],
       [-0.54086814, -1.34210852, -0.49659175],
       [ 0.9534001 ,  0.10313072,  1.67047029],
       [-0.2068231 ,  0.43101148,  0.58157179],
       [-0.31621678, -0.95636993, -0.94369194],
       [-1.1883757 ,  0.81434334,  0.33663351],
       [-0.64549425,  0.28127065,  0.00726466],
       [-0.74445674, -1.70678329,  0.23685874]])

In [278]:
W0 = np.random.randn(1, 3)
W0

array([[ 0.30601856, -1.05976091,  0.16654666]])

## Forward Pass

In [279]:
# Forward pass
def softmax(z):
    exp_z = np.exp(z - np.max(z, axis=1, keepdims=True))
    return exp_z / exp_z.sum(axis=1, keepdims=True)

## Original Evaluation

In [280]:
logits = (X @ W + W0).to_numpy()  # Convert to NumPy array to avoid pandas-specific issues
probs = softmax(logits)          # Predicted class probabilities
preds = np.argmax(probs, axis=1) # Class predictions (0, 1, or 2)

In [281]:
preds

array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 1, 1, 1, 2, 1, 1, 1, 1, 1, 1,
       1, 1, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 2, 1, 1, 1, 1, 1, 1, 1, 1, 2])

In [282]:
accuracy = accuracy_score(y, preds)

print(f"Original Accuracy: {accuracy:.2f}")

Original Accuracy: 0.51


# Training

### Loss Fuction

In [283]:
def cross_entropy_loss(probs, y_true):
    n = y_true.shape[0]
    log_probs = -np.log(probs[range(n), y_true])
    return np.sum(log_probs) / n

### One-hot encoder
class 0 -> [1, 0, 0]

class 1 -> [0, 1, 0]

class 2 -> [0, 0, 1]


In [284]:
def one_hot(y, num_classes):
    return np.eye(num_classes)[y]

In [285]:
learning_rate = 0.1
epochs = 1000
num_classes = 3
n_samples, n_features = X.shape

for epoch in range(epochs):
    # Forward
    logits = np.dot(X, W) + dw0
    probs = softmax(logits)
    loss = cross_entropy_loss(probs, y)

    # Back Propagate
    y_onehot = one_hot(y, num_classes)
    dlogits = (probs - y_onehot) / n_samples
    dW = np.dot(X.T, dlogits)
    dw0 = np.sum(dlogits, axis=0, keepdims=True)

    # Update Weights
    W -= learning_rate * dW
    W0 -= learning_rate * dw0

    # Log
    if epoch % 100 == 0:
        predictions = np.argmax(probs, axis=1)
        acc = np.mean(predictions == y) 
        eval_logits = np.dot(X_test, W) + dw0
        eval_probs = softmax(eval_logits)
        eval_preds = np.argmax(eval_probs, axis=1)
        eval_accuracy = accuracy_score(y_test, eval_preds)
        print(f"Test Accuracy: {accuracy:.2f}")
        print(f"Epoch {epoch}, Loss: {loss:.4f}, Accuracy: {acc:.2f}, Eval Accuracy: {eval_accuracy:.2f}")

Test Accuracy: 0.51
Epoch 0, Loss: 49.8492, Accuracy: 0.51, Eval Accuracy: 0.25
Test Accuracy: 0.51
Epoch 100, Loss: 84.8202, Accuracy: 0.54, Eval Accuracy: 0.30
Test Accuracy: 0.51
Epoch 200, Loss: 94.1574, Accuracy: 0.50, Eval Accuracy: 0.15
Test Accuracy: 0.51
Epoch 300, Loss: 42.5007, Accuracy: 0.30, Eval Accuracy: 0.55
Test Accuracy: 0.51
Epoch 400, Loss: 86.1895, Accuracy: 0.50, Eval Accuracy: 0.10
Test Accuracy: 0.51
Epoch 500, Loss: 39.5213, Accuracy: 0.56, Eval Accuracy: 0.40
Test Accuracy: 0.51
Epoch 600, Loss: 56.3991, Accuracy: 0.60, Eval Accuracy: 0.55
Test Accuracy: 0.51
Epoch 700, Loss: 73.5935, Accuracy: 0.64, Eval Accuracy: 0.45
Test Accuracy: 0.51
Epoch 800, Loss: 28.8833, Accuracy: 0.45, Eval Accuracy: 0.55
Test Accuracy: 0.51
Epoch 900, Loss: 41.4690, Accuracy: 0.59, Eval Accuracy: 0.40


# Evalution

In [286]:
logits = np.dot(X_test, W) + dw0
probs = softmax(logits)
preds = np.argmax(probs, axis=1)
accuracy = accuracy_score(y_test, preds)
print(f"Test Accuracy: {accuracy:.2f}")

Test Accuracy: 0.10


# Inferrence

In [287]:
# Inter Milan
homeTeamName = "Inter Milan"
homeAveragePossesion = 56.5
homeAverageShots = 14.6
homeAverageShotsOnTarget = 4.9
homeAverageFouls = 10.8
homeAverageSaves = 3

# PSG
awayTeamName = "PSG"
awayAveragePossesion = 66.6
awayAverageShots = 18.8
awayAverageShotsOnTarget = 7.9
awayAverageFouls = 10
awayAverageSaves = 3

result_map = {
    0: "Draw",
    1: f"{homeTeamName} Win",
    2: f"{awayTeamName} Win"
}

X = np.array([[homeAveragePossesion, awayAveragePossesion, homeAverageShotsOnTarget, awayAverageShotsOnTarget, homeAverageShots, awayAverageShots, homeAverageFouls, awayAverageFouls, homeAverageSaves, awayAverageSaves]])
logits = np.dot(X, W) + W0
probs = softmax(logits)
preds = np.argmax(probs, axis=1)
print(f"Predicted class: {preds[0]}")
print(f"Predicted result: {result_map[preds[0]]}")

 

Predicted class: 2
Predicted result: PSG Win
